# Troubleshooting
- To troubleshoot I need to know everything about a model at an epoch

# Examine a single word  


In [ ]:
import os
import troubleshooting
import data_wrangling
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
from ipywidgets import interact
from tqdm import tqdm
import pandas as pd
import numpy as np
import altair as alt


# Intactive input/act

In [ ]:
batch_name = "OP_SGD_lr_tune"
code_name = 'OP_SGD_lr_tune_r0009'
testset_name = 'train_r100'
testset = data_wrangling.load_testset(testset_name)
d = troubleshooting.Diagnosis(f"{batch_name}/{code_name}")
clear_output()

In [ ]:
d.eval('train_r100', task="triangle", epoch=200)

# An overall temporal dynamics of the model (HS04 fig. 12)

In [ ]:
class temporal_diagnostic:
    """Examine the temporal dynamic of inputs similar to HS04 fig 12"""

    def __init__(self, diagnosis: troubleshooting.Diagnosis):
        """Remember to do this first before init: d.eval('train_r100', task="triangle", epoch=1000)."""
        self.d = diagnosis
        self.words = self.d.testset_package["item"]
        self.df = self.create_df()

    def get_all_act1(self, word: str, output: str):
        """Get all detailed input diagnoistic in a word at target node == 1"""
        self.d.set_target_word(word, verbose=False)
        if output == "pho":
            return self.d.word_pho_df.loc[d.word_pho_df.target_act == 1]
        elif output == "sem":
            return self.d.word_sem_df.loc[d.word_sem_df.target_act == 1]

    def create_df(self):
        """Create a dataframe for plotting."""
        df_sem = pd.concat(
            [self.get_all_act1(word, "sem") for word in tqdm(self.words)],
            ignore_index=True,
        )
        df_sem["output"] = "sem"
        df_sem = df_sem.loc[df_sem.variable.isin(["PS", "CS", "OS"])]

        df_pho = pd.concat(
            [self.get_all_act1(word, "pho") for word in self.words],
            ignore_index=True,
        )
        df_pho["output"] = "pho"
        df_pho = df_pho.loc[df_pho.variable.isin(["SP", "CP", "OP"])]
        df = pd.concat([df_sem, df_pho], ignore_index=True)
        return df.groupby(["timetick", "variable"]).mean().reset_index()

    def plot(self):
        selection = alt.selection_multi(fields=["variable"], bind="legend")
        return (
            alt.Chart(self.df)
            .mark_line()
            .encode(
                x="timetick:Q",
                y="value:Q",
                color="variable:N",
                opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
            )
            .add_selection(selection)
            .properties(
                title=f"{code_name}: Input temporal dynamic at the end of training among {testset_name}"
            )
        )


In [ ]:
td = temporal_diagnostic(d)


In [ ]:
td.plot()

# Interactive plot to examine a single word

In [ ]:
@interact(
    sel_word=testset['item'], 
    layer=['pho', 'sem'], 
    task=['triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(d.cfg.save_freq, d.cfg.total_number_of_epoch + 1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d.eval(testset_name, task=task, epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer(layer)

# Cooperative vs. competitive relationship

In [ ]:
def get_all_input_over_epoch(word:str):
    """Get all details over epoch in one word at last timetick."""

    df_pho = pd.DataFrame()
    df_sem = pd.DataFrame()

    early_epochs = range(1, 11)
    later_epochs = range(d.cfg.save_freq, 1001, d.cfg.save_freq)
    epochs = list(early_epochs) + list(later_epochs)

    for epoch in tqdm(epochs):

        d.eval(testset_name, task='triangle', epoch=epoch)
        d.set_target_word(word)

        this_pho = d.word_pho_df.loc[d.word_pho_df.timetick == 12]
        this_pho['epoch'] = epoch
        df_pho = df_pho.append(this_pho, ignore_index=True)
        
        this_sem = d.word_sem_df.loc[d.word_sem_df.timetick == 12]
        this_sem['epoch'] = epoch
        df_sem = df_sem.append(this_sem, ignore_index=True)

        clear_output(wait=True)
    
    return df_pho, df_sem

In [ ]:
def get_on_nodes(word:str, representation:str) -> dict:
    """Gets the on nodes of a word in a given representation."""
    w_idx = d.testset_package['item'].index(word)
    p = d.testset_package[representation][w_idx]
    return [x[0] for x in tf.where(p).numpy()]

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

def plot_input_developmental(df: pd.DataFrame, units:list):
    """Plot how input change over epoch in a word"""
    title = f"Development of input over epoch at nodes: {units}"
    df = df.loc[df.unit.isin(units)]

    sel_vars = ['CP', 'OP', 'SP', 'OS', 'CS', 'PS']
    df = df.loc[df.variable.isin(sel_vars)]
    
    selection = alt.selection_multi(fields=['variable'], bind='legend')
    return (
        alt.Chart(df)
        .mark_line()
        .encode(
            x='epoch:Q',
            y='mean(value):Q',
            color='variable',
            opacity=alt.condition(selection, alt.value(1), alt.value(0.2)))
        .add_selection(selection)
        .properties(title=title))

In [ ]:
word = 'close'

df_pho, df_sem = get_all_input_over_epoch(word)
get_on_nodes(word, 'pho')

In [ ]:
off_nodes = set(range(250)).difference(get_on_nodes(word, 'pho'))

In [ ]:
plot_input_developmental(df_pho, off_nodes)

In [ ]:
plot_input_developmental(df_pho, get_on_nodes(word, 'pho'))

In [ ]:
def plot_slot(slot: int):
    """Plotting on and off nodes' input by slot."""
    units = range(slot*25, (slot+1)*25)
    on_nodes = get_on_nodes(word, 'pho')
    on_plot = plot_input_developmental(df_pho, [x for x in units if x in on_nodes])
    off_plot = plot_input_developmental(df_pho, [x for x in units if x not in on_nodes])
    return on_plot | off_plot

plot_slot(0)

In [ ]:
plot_slot(1)

In [ ]:
plot_slot(2)

In [ ]:
plot_slot(3)

In [ ]:
plot_slot(4)

In [ ]:
plot_slot(5)

In [ ]:
plot_slot(6)

In [ ]:
plot_slot(7)

In [ ]:
plot_slot(8)

In [ ]:
plot_slot(9)

In [ ]:
plot_input_developmental(df_sem, get_on_nodes(word, 'sem'))

# Prediction from zero inputs

In [ ]:
import modeling
model = modeling.TriangleModel(cfg)(cfg=d.cfg, batch_size_override=1)
epoch=100
saved_checkpoint = d.cfg.saved_checkpoints_fstring.format(epoch=epoch)

ckpt = tf.train.Checkpoint(model=model)
ckpt.restore(saved_checkpoint).expect_partial()
model.set_active_task('triangle')
zero_inputs = tf.zeros(shape=[1, d.cfg.ort_units])
y_pred = model([zero_inputs] * 12)

In [ ]:
y_pred['input_hop_hp'][-1,:,:].numpy().mean()
y_pred['input_hop_hp'][-1,:,:].numpy().mean()
y_pred['input_hop_hp'][-1,:,:].numpy().mean()

In [ ]:
y_pred['pho']

In [ ]:
y_pred['pho'][12, :, 46]

## Explore Cosine vs. target
Run cosine(sigmoid(input), target) 

In [ ]:
class MyCosine:
    
    def __init__(self, y_true):
        self.y_true = y_true

    @staticmethod
    def sigmoid(x):
        return tf.keras.activations.sigmoid(x)

    def cosine(self, y_pred):
        return -tf.keras.losses.cosine_similarity(self.y_true, y_pred)

    def cosine_on_time_axis(self, ys):
        return tf.map_fn(self.cosine, ys)

    def __call__(self, ys, apply_sigmoid=False):

        if apply_sigmoid:
            ys = self.sigmoid(ys)

        return tf.reduce_mean(self.cosine_on_time_axis(ys), axis=-1).numpy()

    def plot(self, ys, apply_sigmoid=False):
        plt.plot(self.__call__(ys, apply_sigmoid))
        plt.show()

# Instantiating the cosine metric
cos = MyCosine(y_true=d.testset_package['sem'])

### SEM

In [ ]:
ps = cos(d.y_pred['input_hps_hs'], apply_sigmoid=True)
cs = cos(d.y_pred['input_css_cs'], apply_sigmoid=True)
os = cos(d.y_pred['input_hos_hs'], apply_sigmoid=True)
inp = cos(d.y_pred['input_sem'], apply_sigmoid=True)
act_s = cos(d.y_pred['sem'], apply_sigmoid=False)

plt.plot(ps, label='ps')
plt.plot(cs, label='cs')
plt.plot(os, label='os')
plt.plot(inp, label='sum_input')
plt.plot(act_s, label='s')
plt.legend()

### PHO

In [ ]:
cos = MyCosine(y_true=d.testset_package['pho'])

sp = cos(d.y_pred['input_hsp_hp'], apply_sigmoid=True)
cp = cos(d.y_pred['input_cpp_cp'], apply_sigmoid=True)
op = cos(d.y_pred['input_hop_hp'], apply_sigmoid=True)
inp = cos(d.y_pred['input_pho'], apply_sigmoid=True)
act_p = cos(d.y_pred['pho'], apply_sigmoid=False)

plt.plot(sp, label='sp')
plt.plot(cp, label='cp')
plt.plot(op, label='op')
plt.plot(inp, label='sum_input')
plt.plot(act_p, label='p')
plt.legend()